# Análisis de entrevistas PDET
Este notebook permite:
- Leer archivos `.txt` de entrevistas
- Extraer variables como edad y género
- Codificar menciones de violencia
- Realizar análisis estadísticos
- Ajustar un modelo de regresión logística

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
# Especificar la ruta donde estén los .txt
os.chdir('/content/drive/MyDrive/_Luz Stella Doctorado/Mario Chamie Mazzilli/2_entrevistas_txt')
os.getcwd()

'/content/drive/MyDrive/_Luz Stella Doctorado/Mario Chamie Mazzilli/2_entrevistas_txt'

In [ ]:
import os
import pandas as pd
import numpy as np
import re
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from scipy.stats import ttest_ind, mannwhitneyu
import statsmodels.api as sm

In [ ]:
def cargar_entrevistas(directorio):
    data = []
    for archivo in os.listdir(directorio):
        if archivo.endswith('.txt'):
            with open(os.path.join(directorio, archivo), 'r', encoding='utf-8') as f:
                texto = f.read()
                data.append({'archivo': archivo, 'contenido': texto})
    return pd.DataFrame(data)

In [ ]:
"""
ESTE CÓDIGO FALLO
def extraer_variables(df):
    def extraer_valor(texto, etiqueta):
        match = re.search(rf'{etiqueta}:(.*)', texto)
        return match.group(1).strip() if match else np.nan
    df['edad'] = df['contenido'].apply(lambda x: extraer_valor(x, 'Edad'))
    df['genero'] = df['contenido'].apply(lambda x: extraer_valor(x, 'Género'))
    df['sector'] = df['contenido'].apply(lambda x: extraer_valor(x, 'Entidad|Organización|Institución|Municipio'))
    return df
"""

"\nESTE CÓDIGO FALLO\ndef extraer_variables(df):\n    def extraer_valor(texto, etiqueta):\n        match = re.search(rf'{etiqueta}:(.*)', texto)\n        return match.group(1).strip() if match else np.nan\n    df['edad'] = df['contenido'].apply(lambda x: extraer_valor(x, 'Edad'))\n    df['genero'] = df['contenido'].apply(lambda x: extraer_valor(x, 'Género'))\n    df['sector'] = df['contenido'].apply(lambda x: extraer_valor(x, 'Entidad|Organización|Institución|Municipio'))\n    return df\n"

In [ ]:
def extraer_variables(df):
    def extraer_valor(texto, etiquetas):
        for etiqueta in etiquetas:
            match = re.search(rf'{etiqueta}:(.*)', texto)
            if match:
                return match.group(1).strip()
        return np.nan

    df['edad'] = df['contenido'].apply(lambda x: extraer_valor(x, ['Edad']))
    df['genero'] = df['contenido'].apply(lambda x: extraer_valor(x, ['Género']))
    df['sector'] = df['contenido'].apply(lambda x: extraer_valor(x, [
        'Entidad', 'Organización', 'Institución', 'Municipio'
    ]))
    return df


In [ ]:
def codificar_violencia(df):
    patrones_violencia = ['violencia', 'conflicto', 'amenaza', 'desplazamiento', 'reclutamiento']
    df['violencia'] = df['contenido'].apply(
        lambda x: int(any(pat in x.lower() for pat in patrones_violencia))
    )
    return df

In [ ]:
def pruebas_estadisticas(df):
    df['genero'] = df['genero'].astype('category')
    grupos = df.groupby('genero')['violencia'].mean()
    print("Proporción de violencia por género:")
    print(grupos)
    print("\nPrueba t de Student:")
    hombres = df[df['genero'] == 'M']['violencia']
    mujeres = df[df['genero'] == 'F']['violencia']
    print(ttest_ind(hombres, mujeres, equal_var=False))

In [ ]:
def regresion_logistica(df):
    df_modelo = df[['edad', 'genero', 'violencia']].dropna()
    df_modelo['edad'] = df_modelo['edad'].astype(float)
    df_modelo['genero'] = LabelEncoder().fit_transform(df_modelo['genero'])
    X = df_modelo[['edad', 'genero']]
    y = df_modelo['violencia']
    modelo = sm.Logit(y, sm.add_constant(X)).fit()
    print(modelo.summary())

In [ ]:
"""
FALLA SI NO HAY VARIACIÓN EN LA VARIABLE DEPENDIENTE
# Cambia esta ruta a la ubicación de tus archivos .txt en Google Drive
directorio_txt = "/content/drive/MyDrive/_Luz Stella Doctorado/Mario Chamie Mazzilli/2_entrevistas_txt"
df_entrevistas = cargar_entrevistas(directorio_txt)
df_entrevistas = extraer_variables(df_entrevistas)
df_entrevistas = codificar_violencia(df_entrevistas)
pruebas_estadisticas(df_entrevistas)
regresion_logistica(df_entrevistas)
"""

'\nFALLA SI NO HAY VARIACIÓN EN LA VARIABLE DEPENDIENTE\n# Cambia esta ruta a la ubicación de tus archivos .txt en Google Drive\ndirectorio_txt = "/content/drive/MyDrive/_Luz Stella Doctorado/Mario Chamie Mazzilli/2_entrevistas_txt"\ndf_entrevistas = cargar_entrevistas(directorio_txt)\ndf_entrevistas = extraer_variables(df_entrevistas)\ndf_entrevistas = codificar_violencia(df_entrevistas)\npruebas_estadisticas(df_entrevistas)\nregresion_logistica(df_entrevistas)\n'

In [ ]:
"""
FALLA POR COLINEALIDAD
def regresion_logistica(df):
    df_modelo = df[['edad', 'genero', 'violencia']].dropna()
    df_modelo['edad'] = df_modelo['edad'].astype(float)
    df_modelo['genero'] = LabelEncoder().fit_transform(df_modelo['genero'])

    if df_modelo['violencia'].nunique() < 2:
        print("⚠️ No se puede ajustar el modelo: todos los casos tienen la misma clase en la variable 'violencia'.")
        return

    X = df_modelo[['edad', 'genero']]
    y = df_modelo['violencia']
    modelo = sm.Logit(y, sm.add_constant(X)).fit()
    print(modelo.summary())
"""

In [ ]:
def regresion_logistica(df):
    df_modelo = df[['edad', 'genero', 'violencia']].dropna()
    df_modelo['edad'] = df_modelo['edad'].astype(float)
    df_modelo['genero'] = LabelEncoder().fit_transform(df_modelo['genero'])

    if df_modelo['violencia'].nunique() < 2:
        print("⚠️ No se puede ajustar el modelo: todos los casos tienen la misma clase en la variable 'violencia'.")
        return

    if df_modelo.groupby(['genero', 'violencia']).size().min() == 0:
        print("⚠️ No se puede ajustar el modelo: hay combinación perfecta entre variables predictoras y respuesta.")
        return

    X = df_modelo[['edad', 'genero']]
    y = df_modelo['violencia']
    modelo = sm.Logit(y, sm.add_constant(X)).fit()
    print(modelo.summary())

### ✅ ¿Qué hace el bloque que sigue?

* Carga entrevistas desde su ruta personalizada.
* Extrae variables de interés (`edad`, `genero`, `sector`).
* Codifica si hay violencia.
* Muestra los resultados intermedios y finales de:

  * **Distribuciones por género**
  * **Prueba t de Student**
  * **Modelo de regresión logística**


In [ ]:
# Paso 1: Definir la ruta a la carpeta en Google Drive con los .txt
directorio_txt = "/content/drive/MyDrive/_Luz Stella Doctorado/Mario Chamie Mazzilli/2_entrevistas_txt"  # Reemplazar con su ruta real

# Paso 2: Ejecutar el flujo de procesamiento
df_entrevistas = cargar_entrevistas(directorio_txt)
df_entrevistas = extraer_variables(df_entrevistas)
df_entrevistas = codificar_violencia(df_entrevistas)

# Mostrar una vista previa del DataFrame con variables extraídas
print("🔎 Vista previa del DataFrame:\n")
display(df_entrevistas[['archivo', 'edad', 'genero', 'sector', 'violencia']])

# Paso 3: Pruebas estadísticas
print("\n📊 Resultados de las pruebas estadísticas:\n")
pruebas_estadisticas(df_entrevistas)

# Paso 4: Regresión logística
print("\n📈 Resultados del modelo de regresión logística:\n")
regresion_logistica(df_entrevistas)


🔎 Vista previa del DataFrame:



,archivo,edad,genero,sector,violencia
0,entrevista_03.txt,50,M,Cooperación Internacional,1
1,entrevista_04.txt,29,F,Comunidad Indígena Zenú,1
2,entrevista_01.txt,45,M,Alcaldía Municipal de El Carmen,1
3,entrevista_02.txt,34,F,Gobernación de Bolívar,0
4,entrevista_06.txt,38,M,Corporación Educativa Regional,1
5,entrevista_05.txt,40,F,Fundación Cultural de Paz,0



📊 Resultados de las pruebas estadísticas:

Proporción de violencia por género:
genero
F    0.333333
M    1.000000
Name: violencia, dtype: float64

Prueba t de Student:
TtestResult(statistic=np.float64(2.0), pvalue=np.float64(0.1835034190722739), df=np.float64(2.0))

📈 Resultados del modelo de regresión logística:

         Current function value: 0.000000
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:              violencia   No. Observations:                    6
Model:                          Logit   Df Residuals:                        3
Method:                           MLE   Df Model:                            2
Date:                Thu, 05 Jun 2025   Pseudo R-squ.:                   1.000
Time:                        19:11:30   Log-Likelihood:            -2.3647e-09
converged:                      False   LL-Null:                       -3.8191
Covariance Type:            nonrobust   LLR p-value:            

<ipython-input-39-c4adb50c6ca8>:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grupos = df.groupby('genero')['violencia'].mean()
/usr/local/lib/python3.11/dist-packages/scipy/stats/_axis_nan_policy.py:586: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/usr/local/lib/python3.11/dist-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/usr/local/lib/python3.11/dist-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction de